In [ ]:
import pandas as pd
import numpy as np
import pandas as pd
import numpy as np
import warnings
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import statistics
from sklearn.linear_model import ElasticNet

import plotly.figure_factory as ff
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
from sklearn.exceptions import ConvergenceWarning
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=ConvergenceWarning)


In [ ]:
df=pd.read_csv("/content/sample_data/conversion_data_train.csv")

In [ ]:
df.head()

,country,age,new_user,source,total_pages_visited,converted
0,China,22,1,Direct,2,0
1,UK,21,1,Ads,3,0
2,Germany,20,0,Seo,14,1
3,US,23,1,Seo,3,0
4,US,28,1,Direct,3,0


In [ ]:
df.describe(include="all")

,country,age,new_user,source,total_pages_visited,converted
count,284580,284580.000000,284580.000000,284580,284580.000000,284580.000000
unique,4,NaN,NaN,3,NaN,NaN
top,US,NaN,NaN,Seo,NaN,NaN
freq,160124,NaN,NaN,139477,NaN,NaN
mean,NaN,30.564203,0.685452,NaN,4.873252,0.032258
std,NaN,8.266789,0.464336,NaN,3.341995,0.176685
min,NaN,17.000000,0.000000,NaN,1.000000,0.000000
25%,NaN,24.000000,0.000000,NaN,2.000000,0.000000
50%,NaN,30.000000,1.000000,NaN,4.000000,0.000000
75%,NaN,36.000000,1.000000,NaN,7.000000,0.000000


The target variable is "converted", which is categorial and takes 2 values: 0 and 1. There are 5 explanative variables: 3 categorical ("country" taking 4 values, "new_user" taking 2 values, and "source" taking 3 values) and 2 numerical ("age" from 17 to 123 years old and "total_pages_visited" from 1 to 29 pages). Let's remove ages above 100 years old, which seems quite weird.

In [ ]:
df = df[df['age'] <= 100]

In [ ]:
print(df.shape)
df.isnull().sum()/len(df)
#No missing values

(284578, 6)


country                0.0
age                    0.0
new_user               0.0
source                 0.0
total_pages_visited    0.0
converted              0.0
dtype: float64

In [ ]:
df['converted'].value_counts()

0    275400
1      9178
Name: converted, dtype: int64

In [ ]:
#Observation of the distribution

num_features = ['age','total_pages_visited']

for feature in num_features:
    fig = px.histogram(df, feature, color = 'converted', facet_row = 'converted', histnorm = 'probability')
    fig.show()

We already observe a tendency that people who suscribe are a bit younger in general that people who don't, and that the number of pages visited before suscription is elevated (approximateky 10 to 20) whereas people who don't suscribe usually don't hesitate, they just visit less than 5-10 pages.

In [ ]:
# Group the original DataFrame by 'country' and calculate the sum
df_country = df.groupby(df['country']).sum()

# Add columns with the counts of each country
df_country['Population'] = df.groupby('country')['age'].count()

#Percentage of "new_users" per country
df_country["Percentage_new_users"]=df_country['new_user']/df_country["Population"]

#Percentage of "Converted" per country
df_country["Percentage converted per country"]=df_country['converted']/df_country["Population"]

# Reset the index to make 'country' a regular column
df_country.reset_index(inplace=True)

# Display the updated DataFrame
df_country.head()

<ipython-input-9-1d0afea711d5>:2: FutureWarning:

The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.



,country,age,new_user,total_pages_visited,converted,Population,Percentage_new_users,Percentage converted per country
0,China,2119558,48325,314616,89,69122,0.699126,0.001288
1,Germany,356022,7928,60878,729,11692,0.678070,0.062350
2,UK,1328045,29613,221642,2290,43640,0.678575,0.052475
3,US,4894102,109200,789669,6070,160124,0.681971,0.037908


In [ ]:
# Extract the numeric columns (excluding 'country')
numeric_columns = df_country[["new_user", "converted", "Population","Percentage_new_users","Percentage converted per country"]]

# Define the number of columns in your subplot
num_columns = len(numeric_columns.columns)

# Create subplots
fig = make_subplots(rows=1, cols=num_columns, subplot_titles=numeric_columns.columns)

# Loop through the numeric columns and create bar charts in subplots
for i, column in enumerate(numeric_columns.columns):
    bar_chart = go.Bar(x=df_country['country'], y=df_country[column], marker_color='blue')
    fig.add_trace(bar_chart, row=1, col=i+1)
    fig.update_xaxes(title_text='Country', row=1, col=i+1)
    fig.update_yaxes(title_text='Value', row=1, col=i+1)
    fig.update_xaxes(categoryorder='total ascending', row=1, col=i+1)
    fig.update_xaxes(tickangle=45, row=1, col=i+1)
    fig.update_layout( showlegend=False)

# Adjust subplot layout
fig.update_layout(width=1500, height=500)

# Show the figure
fig.show()


Those graphs shows that the newsletter companie only have visitors from the US (160.124k),China (69.122k), UK (43.64K) and Germany.And it seems they can only reach those 4 countries.

Among those country the sum new_user mostly comes from USA (109.2K), then China (48.325K), UK (29.613K) and last Germany (7,928).
Furthermore, the percentage_new_users graph shows the about same ratio of new users between 67%-69%, which is pretty good.

However, the convertion rate is low and has a lot of disparities:
- Germany : 6.2%
- UK : 5.2%
- US 3.8%
- China 0.12%

Considering the percentage of new_users (67%-69%) and the mean percentage of "converted" across all the country (3,85%), we believe that a study needs to be conducted to understand why the "new_users" aren't subscribing to the newsletter.

In [ ]:
# Group by 'source' and calculate the sum for each source
df_source = df.groupby('source').sum().reset_index()

# Define a list of countries you want to count
countries = ['China', 'Germany', 'UK', 'US']

# Initialize an empty dictionary to store the country-specific counts
country_counts = {}

# Loop through the countries and count rows in each 'source' group
for country in countries:
    counts = df[df['country'] == country].groupby('source').size().reset_index(name=country)
    country_counts[country] = counts

# Merge the DataFrames sequentially based on 'source'
for country, counts_df in country_counts.items():
    df_source = pd.merge(df_source, counts_df, on='source', how='left')


df_source.head()


<ipython-input-11-527486de5541>:2: FutureWarning:

The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.



,source,age,new_user,total_pages_visited,converted,China,Germany,UK,US
0,Ads,2443118,54932,392217,2784,19519,3379,12169,44959
1,Direct,1988105,44652,313373,1810,15724,2562,10021,36769
2,Seo,4266504,95482,681215,4584,33879,5751,21450,78396


In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Exclude the 'age' column
exclude_columns = ['age']
source_columns = df_source.select_dtypes(include='number').drop(exclude_columns, axis=1)

# Define the number of columns in your subplot
num_columns = len(source_columns.columns)

# Create subplots with custom specs
specs = [[{"type": "pie"} for _ in range(num_columns)]]

fig_pie = make_subplots(rows=1, cols=num_columns, specs=specs, subplot_titles=source_columns.columns)

# Loop through the numeric columns and create pie charts in subplots
for i, column in enumerate(source_columns.columns):
    column_data = source_columns[column]
    labels = column_data.index
    values = column_data.values

    pie_chart = go.Pie(labels=df_source['source'], values=values,
                       texttemplate = "%{label}<br>(%{percent})",
                       textposition = "inside")

    fig_pie.add_trace(pie_chart, row=1, col=i+1)

    fig_pie.update_layout(showlegend=False)

# Adjust subplot layout
fig_pie.update_layout(width=1500, height=500)

# Show the figure
fig_pie.show()


As we can see above, SEO is the way to go with the best results in each country for the sum of new_user and the convertion.

It also seems like the people coming to the website via SEO tends to visit more pages compare to direct recommendation and ads. Thus SEO bring qualitatives readers for an higher number of convertion to the newsletter

In [ ]:
df_converted = df.groupby(df['converted']).sum()
df_converted.reset_index(inplace=True)
df_converted=df_converted.drop(['age'], axis=1)
df_converted.head()

<ipython-input-13-14e0e2a42fb2>:1: FutureWarning:

The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.



,converted,new_user,total_pages_visited
0,0,192325,1253169
1,1,2741,133636


In [ ]:
# Extract the numeric columns (excluding 'country')
numeric_columns_converted = df_converted[["new_user", "total_pages_visited"]]

# Define the number of columns in your subplot
num_columns_converted = len(numeric_columns_converted.columns)
title = numeric_columns_converted.columns
subplot_titles = [f'Count of converted per {col} ' for col in title]values

# Create subplots
fig_converted = make_subplots(rows=1, cols=num_columns_converted, subplot_titles=subplot_titles)

# Loop through the numeric columns and create bar charts in subplots
for i, column in enumerate(numeric_columns_converted.columns):
    bar_chart_converted = go.Bar(x=df_converted['converted'], y=df_converted[column] )
    fig_converted.add_trace(bar_chart_converted, row=1, col=i+1)
    fig_converted.update_xaxes(title_text='Converted', row=1, col=i+1)
    fig_converted.update_yaxes(title_text='Count', row=1, col=i+1)
    fig_converted.update_xaxes(categoryorder='total ascending', row=1, col=i+1)
    fig_converted.update_layout(showlegend=False)

# Adjust subplot layout
fig_converted.update_layout(width=1500, height=500)

# Show the figure
fig_converted.show()


In [ ]:
corr_matrix = df.corr().round(2)
corr_matrix

<ipython-input-15-bac0748edce0>:1: FutureWarning:

The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.



,age,new_user,total_pages_visited,converted
age,1.00,0.01,-0.05,-0.09
new_user,0.01,1.00,-0.08,-0.15
total_pages_visited,-0.05,-0.08,1.00,0.53
converted,-0.09,-0.15,0.53,1.00


In [ ]:
#Correlation matrix for numerical values

fig_corr = ff.create_annotated_heatmap(corr_matrix.values,
                                  x = corr_matrix.columns.tolist(),
                                  y = corr_matrix.index.tolist(),
                                  )
fig_corr.update_layout(title_text='Correlation Heatmap')
fig_corr.show()

The correlation matrix shows that the total_pages_visited is the most correlated with our target variable "converted" (positive correlation at 0.53).